## License
This file is part of the project megFingerprinting. All of megFingerprinting code is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version. megFingerprinting is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details. You should have received a copy of the GNU General Public License along with megFingerprinting. If not, see https://www.gnu.org/licenses/.

In [2]:
import difflib
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
import seaborn as sns
import scipy as sp
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.stats import pearsonr
sns.set(font_scale=2)
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("husl", 8))

/Users/jasondsc/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), sp.stats.sem(a)
    h = np.percentile(a, (1-((1-confidence)/2))*100)
    l = np.percentile(a, ((1-confidence)/2)*100)
    return m, l, h



# I. Subject Identifiability: Original Feature Space vs PCA Reconstructed

In [8]:
# Yeo network definition 
dmn=[0,4,14,15,20,21,30,31,34,38,39,40,50,51,52,53,56,57]
da=[58, 59]
fp= [5,54, 55]
limbic= [8,9,10,11,16,17,24,25,28,29,64,65]
som=[1,32,33,44,45,48,49,60,61,66,67]
va=[2,3,18,19,36,37,41,46,47,62,63]
vis=[6,7,12,13,22,23,26,27,35,42,43]

lista= [dmn,da,fp,limbic,som,va,vis]
listb= [18,2,3,12,11,11,11]



In [10]:
# change in AIC per Yeo network
self_id_roi = np.zeros((7,154))
accuracy_roi=np.zeros((2,7))

# Warangle data set into two big feature matrices
def prune_subject_csv(filename, ind):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[ind,0:451].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat

for i in range(7):
    ind=lista[i]
    # Parameters
    n_subs = 154 # Change here to get number of participants! 
    n_feats = int(listb[i]*451)
    n_measurements = 2
    # Get n subjects: both training and testing datasets
    onlyfiles = [f for f in listdir('NEWspectraFUL/') if isfile(join('NEWspectraFUL/', f))]
    sub_target = np.zeros([n_subs, (n_feats)+1])
    sub_database = np.zeros([n_subs, (n_feats)+1])
    iv = 0
    it = 0
    for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
        sub = 'NEWspectraFUL/' + iFile
        print(sub)
        if sub[33] == 'v':
            sub_target[iv, :] = prune_subject_csv(sub, ind)
            iv += 1
        else:
            sub_database[it, :] = prune_subject_csv(sub, ind)
            it += 1
    # Correlations can be computed as the dot product between two z-scored vectors
    z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
    z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
    predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
    target_from_database = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
    database_from_target = accuracy_score(range(n_subs), predictions.argmax(axis = 0))

    print('When predicting the target from the database, we get a ' + str(target_from_database*100)[0:5] + '% accuracy')
    print('When predicting the database from the target, we get a ' + str(database_from_target*100)[0:5] + '% accuracy')
    
    accuracy_roi[0,i]=target_from_database*100
    accuracy_roi[1,i]=database_from_target*100
    # For the figure, we also get self-identifiability and reconstructed self-identifiability
    self_id_roi[i,:]= np.diagonal(sp.stats.zscore(predictions, axis = 1))

print(self_id_roi)
        

NEWspectraFUL/sub_00001_spectrum_training.csv
0001
NEWspectraFUL/sub_00001_spectrum_validation.csv
0001
NEWspectraFUL/sub_00002_spectrum_training.csv
0002
NEWspectraFUL/sub_00002_spectrum_validation.csv
0002
NEWspectraFUL/sub_00003_spectrum_training.csv
0003
NEWspectraFUL/sub_00003_spectrum_validation.csv
0003
NEWspectraFUL/sub_00004_spectrum_training.csv
0004
NEWspectraFUL/sub_00004_spectrum_validation.csv
0004
NEWspectraFUL/sub_00005_spectrum_training.csv
0005
NEWspectraFUL/sub_00005_spectrum_validation.csv
0005
NEWspectraFUL/sub_00006_spectrum_training.csv
0006
NEWspectraFUL/sub_00006_spectrum_validation.csv
0006
NEWspectraFUL/sub_00007_spectrum_training.csv
0007
NEWspectraFUL/sub_00007_spectrum_validation.csv
0007
NEWspectraFUL/sub_00008_spectrum_training.csv
0008
NEWspectraFUL/sub_00008_spectrum_validation.csv
0008
NEWspectraFUL/sub_00009_spectrum_training.csv
0009
NEWspectraFUL/sub_00009_spectrum_validation.csv
0009
NEWspectraFUL/sub_00010_spectrum_training.csv
0010
NEWspectraFUL

NEWspectraFUL/sub_00081_spectrum_training.csv
0081
NEWspectraFUL/sub_00081_spectrum_validation.csv
0081
NEWspectraFUL/sub_00082_spectrum_training.csv
0082
NEWspectraFUL/sub_00082_spectrum_validation.csv
0082
NEWspectraFUL/sub_00083_spectrum_training.csv
0083
NEWspectraFUL/sub_00083_spectrum_validation.csv
0083
NEWspectraFUL/sub_00084_spectrum_training.csv
0084
NEWspectraFUL/sub_00084_spectrum_validation.csv
0084
NEWspectraFUL/sub_00085_spectrum_training.csv
0085
NEWspectraFUL/sub_00085_spectrum_validation.csv
0085
NEWspectraFUL/sub_00086_spectrum_training.csv
0086
NEWspectraFUL/sub_00086_spectrum_validation.csv
0086
NEWspectraFUL/sub_00087_spectrum_training.csv
0087
NEWspectraFUL/sub_00087_spectrum_validation.csv
0087
NEWspectraFUL/sub_00088_spectrum_training.csv
0088
NEWspectraFUL/sub_00088_spectrum_validation.csv
0088
NEWspectraFUL/sub_00089_spectrum_training.csv
0089
NEWspectraFUL/sub_00089_spectrum_validation.csv
0089
NEWspectraFUL/sub_00090_spectrum_training.csv
0090
NEWspectraFUL

NEWspectraFUL/sub_00005_spectrum_training.csv
0005
NEWspectraFUL/sub_00005_spectrum_validation.csv
0005
NEWspectraFUL/sub_00006_spectrum_training.csv
0006
NEWspectraFUL/sub_00006_spectrum_validation.csv
0006
NEWspectraFUL/sub_00007_spectrum_training.csv
0007
NEWspectraFUL/sub_00007_spectrum_validation.csv
0007
NEWspectraFUL/sub_00008_spectrum_training.csv
0008
NEWspectraFUL/sub_00008_spectrum_validation.csv
0008
NEWspectraFUL/sub_00009_spectrum_training.csv
0009
NEWspectraFUL/sub_00009_spectrum_validation.csv
0009
NEWspectraFUL/sub_00010_spectrum_training.csv
0010
NEWspectraFUL/sub_00010_spectrum_validation.csv
0010
NEWspectraFUL/sub_00011_spectrum_training.csv
0011
NEWspectraFUL/sub_00011_spectrum_validation.csv
0011
NEWspectraFUL/sub_00012_spectrum_training.csv
0012
NEWspectraFUL/sub_00012_spectrum_validation.csv
0012
NEWspectraFUL/sub_00013_spectrum_training.csv
0013
NEWspectraFUL/sub_00013_spectrum_validation.csv
0013
NEWspectraFUL/sub_00014_spectrum_training.csv
0014
NEWspectraFUL

NEWspectraFUL/sub_00084_spectrum_training.csv
0084
NEWspectraFUL/sub_00084_spectrum_validation.csv
0084
NEWspectraFUL/sub_00085_spectrum_training.csv
0085
NEWspectraFUL/sub_00085_spectrum_validation.csv
0085
NEWspectraFUL/sub_00086_spectrum_training.csv
0086
NEWspectraFUL/sub_00086_spectrum_validation.csv
0086
NEWspectraFUL/sub_00087_spectrum_training.csv
0087
NEWspectraFUL/sub_00087_spectrum_validation.csv
0087
NEWspectraFUL/sub_00088_spectrum_training.csv
0088
NEWspectraFUL/sub_00088_spectrum_validation.csv
0088
NEWspectraFUL/sub_00089_spectrum_training.csv
0089
NEWspectraFUL/sub_00089_spectrum_validation.csv
0089
NEWspectraFUL/sub_00090_spectrum_training.csv
0090
NEWspectraFUL/sub_00090_spectrum_validation.csv
0090
NEWspectraFUL/sub_00091_spectrum_training.csv
0091
NEWspectraFUL/sub_00091_spectrum_validation.csv
0091
NEWspectraFUL/sub_00092_spectrum_training.csv
0092
NEWspectraFUL/sub_00092_spectrum_validation.csv
0092
NEWspectraFUL/sub_00093_spectrum_training.csv
0093
NEWspectraFUL

NEWspectraFUL/sub_00008_spectrum_validation.csv
0008
NEWspectraFUL/sub_00009_spectrum_training.csv
0009
NEWspectraFUL/sub_00009_spectrum_validation.csv
0009
NEWspectraFUL/sub_00010_spectrum_training.csv
0010
NEWspectraFUL/sub_00010_spectrum_validation.csv
0010
NEWspectraFUL/sub_00011_spectrum_training.csv
0011
NEWspectraFUL/sub_00011_spectrum_validation.csv
0011
NEWspectraFUL/sub_00012_spectrum_training.csv
0012
NEWspectraFUL/sub_00012_spectrum_validation.csv
0012
NEWspectraFUL/sub_00013_spectrum_training.csv
0013
NEWspectraFUL/sub_00013_spectrum_validation.csv
0013
NEWspectraFUL/sub_00014_spectrum_training.csv
0014
NEWspectraFUL/sub_00014_spectrum_validation.csv
0014
NEWspectraFUL/sub_00015_spectrum_training.csv
0015
NEWspectraFUL/sub_00015_spectrum_validation.csv
0015
NEWspectraFUL/sub_00016_spectrum_training.csv
0016
NEWspectraFUL/sub_00016_spectrum_validation.csv
0016
NEWspectraFUL/sub_00017_spectrum_training.csv
0017
NEWspectraFUL/sub_00017_spectrum_validation.csv
0017
NEWspectraF

...

NEWspectraFUL/sub_00095_spectrum_validation.csv
0095
NEWspectraFUL/sub_00096_spectrum_training.csv
0096
NEWspectraFUL/sub_00096_spectrum_validation.csv
0096
NEWspectraFUL/sub_00097_spectrum_training.csv
0097
NEWspectraFUL/sub_00097_spectrum_validation.csv
0097
NEWspectraFUL/sub_00098_spectrum_training.csv
0098
NEWspectraFUL/sub_00098_spectrum_validation.csv
0098
NEWspectraFUL/sub_00099_spectrum_training.csv
0099
NEWspectraFUL/sub_00099_spectrum_validation.csv
0099
NEWspectraFUL/sub_00100_spectrum_training.csv
0100
NEWspectraFUL/sub_00100_spectrum_validation.csv
0100
NEWspectraFUL/sub_00101_spectrum_training.csv
0101
NEWspectraFUL/sub_00101_spectrum_validation.csv
0101
NEWspectraFUL/sub_00102_spectrum_training.csv
0102
NEWspectraFUL/sub_00102_spectrum_validation.csv
0102
NEWspectraFUL/sub_00103_spectrum_training.csv
0103
NEWspectraFUL/sub_00103_spectrum_validation.csv
0103
NEWspectraFUL/sub_00104_spectrum_training.csv
0104
NEWspectraFUL/sub_00104_spectrum_validation.csv
0104
NEWspectraF

NEWspectraFUL/sub_00020_spectrum_training.csv
0020
NEWspectraFUL/sub_00020_spectrum_validation.csv
0020
NEWspectraFUL/sub_00021_spectrum_training.csv
0021
NEWspectraFUL/sub_00021_spectrum_validation.csv
0021
NEWspectraFUL/sub_00022_spectrum_training.csv
0022
NEWspectraFUL/sub_00022_spectrum_validation.csv
0022
NEWspectraFUL/sub_00023_spectrum_training.csv
0023
NEWspectraFUL/sub_00023_spectrum_validation.csv
0023
NEWspectraFUL/sub_00024_spectrum_training.csv
0024
NEWspectraFUL/sub_00024_spectrum_validation.csv
0024
NEWspectraFUL/sub_00025_spectrum_training.csv
0025
NEWspectraFUL/sub_00025_spectrum_validation.csv
0025
NEWspectraFUL/sub_00026_spectrum_training.csv
0026
NEWspectraFUL/sub_00026_spectrum_validation.csv
0026
NEWspectraFUL/sub_00027_spectrum_training.csv
0027
NEWspectraFUL/sub_00027_spectrum_validation.csv
0027
NEWspectraFUL/sub_00028_spectrum_training.csv
0028
NEWspectraFUL/sub_00028_spectrum_validation.csv
0028
NEWspectraFUL/sub_00029_spectrum_training.csv
0029
NEWspectraFUL

NEWspectraFUL/sub_00099_spectrum_training.csv
0099
NEWspectraFUL/sub_00099_spectrum_validation.csv
0099
NEWspectraFUL/sub_00100_spectrum_training.csv
0100
NEWspectraFUL/sub_00100_spectrum_validation.csv
0100
NEWspectraFUL/sub_00101_spectrum_training.csv
0101
NEWspectraFUL/sub_00101_spectrum_validation.csv
0101
NEWspectraFUL/sub_00102_spectrum_training.csv
0102
NEWspectraFUL/sub_00102_spectrum_validation.csv
0102
NEWspectraFUL/sub_00103_spectrum_training.csv
0103
NEWspectraFUL/sub_00103_spectrum_validation.csv
0103
NEWspectraFUL/sub_00104_spectrum_training.csv
0104
NEWspectraFUL/sub_00104_spectrum_validation.csv
0104
NEWspectraFUL/sub_00105_spectrum_training.csv
0105
NEWspectraFUL/sub_00105_spectrum_validation.csv
0105
NEWspectraFUL/sub_00106_spectrum_training.csv
0106
NEWspectraFUL/sub_00106_spectrum_validation.csv
0106
NEWspectraFUL/sub_00107_spectrum_training.csv
0107
NEWspectraFUL/sub_00107_spectrum_validation.csv
0107
NEWspectraFUL/sub_00108_spectrum_training.csv
0108
NEWspectraFUL

NEWspectraFUL/sub_00022_spectrum_validation.csv
0022
NEWspectraFUL/sub_00023_spectrum_training.csv
0023
NEWspectraFUL/sub_00023_spectrum_validation.csv
0023
NEWspectraFUL/sub_00024_spectrum_training.csv
0024
NEWspectraFUL/sub_00024_spectrum_validation.csv
0024
NEWspectraFUL/sub_00025_spectrum_training.csv
0025
NEWspectraFUL/sub_00025_spectrum_validation.csv
0025
NEWspectraFUL/sub_00026_spectrum_training.csv
0026
NEWspectraFUL/sub_00026_spectrum_validation.csv
0026
NEWspectraFUL/sub_00027_spectrum_training.csv
0027
NEWspectraFUL/sub_00027_spectrum_validation.csv
0027
NEWspectraFUL/sub_00028_spectrum_training.csv
0028
NEWspectraFUL/sub_00028_spectrum_validation.csv
0028
NEWspectraFUL/sub_00029_spectrum_training.csv
0029
NEWspectraFUL/sub_00029_spectrum_validation.csv
0029
NEWspectraFUL/sub_00030_spectrum_training.csv
0030
NEWspectraFUL/sub_00030_spectrum_validation.csv
0030
NEWspectraFUL/sub_00031_spectrum_training.csv
0031
NEWspectraFUL/sub_00031_spectrum_validation.csv
0031
NEWspectraF

NEWspectraFUL/sub_00102_spectrum_validation.csv
0102
NEWspectraFUL/sub_00103_spectrum_training.csv
0103
NEWspectraFUL/sub_00103_spectrum_validation.csv
0103
NEWspectraFUL/sub_00104_spectrum_training.csv
0104
NEWspectraFUL/sub_00104_spectrum_validation.csv
0104
NEWspectraFUL/sub_00105_spectrum_training.csv
0105
NEWspectraFUL/sub_00105_spectrum_validation.csv
0105
NEWspectraFUL/sub_00106_spectrum_training.csv
0106
NEWspectraFUL/sub_00106_spectrum_validation.csv
0106
NEWspectraFUL/sub_00107_spectrum_training.csv
0107
NEWspectraFUL/sub_00107_spectrum_validation.csv
0107
NEWspectraFUL/sub_00108_spectrum_training.csv
0108
NEWspectraFUL/sub_00108_spectrum_validation.csv
0108
NEWspectraFUL/sub_00109_spectrum_training.csv
0109
NEWspectraFUL/sub_00109_spectrum_validation.csv
0109
NEWspectraFUL/sub_00110_spectrum_training.csv
0110
NEWspectraFUL/sub_00110_spectrum_validation.csv
0110
NEWspectraFUL/sub_00111_spectrum_training.csv
0111
NEWspectraFUL/sub_00111_spectrum_validation.csv
0111
NEWspectraF

In [11]:

df = pd.DataFrame(self_id_roi)
df.to_csv("AIC_network_psd_test.csv")

df = pd.DataFrame(accuracy_roi)
df.to_csv("AIC_network_psd_test_acc.csv")


In [12]:
# change in AIC per ROI
def prune_subject_csv(filename, roi):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    #print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[np.arange(68) != roi,0:451].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat

# Parameters
n_subs = 154 # Change here to get number of participants! 
n_feats = int(67*451)
n_measurements = 2
self_id_roi = np.zeros((68,154))
accuracy_roi=np.zeros((2,68))
# Get n subjects: both training and testing datasets
for i in range(68):
    onlyfiles = [f for f in listdir('NEWspectraFUL/') if isfile(join('NEWspectraFUL/', f))]
    sub_target = np.zeros([n_subs, (n_feats)+1])
    sub_database = np.zeros([n_subs, (n_feats)+1])
    iv = 0
    it = 0
    for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
        sub = 'NEWspectraFUL/' + iFile
        #print(sub)
        #print(sub[33])
        if sub[33] == 'v':
            sub_target[iv, :] = prune_subject_csv(sub, i)
            iv += 1
        else:
            sub_database[it, :] = prune_subject_csv(sub, i)
            it += 1

    # Correlations can be computed as the dot product between two z-scored vectors
    z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
    z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
    predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
    target_from_database = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
    database_from_target = accuracy_score(range(n_subs), predictions.argmax(axis = 0))

    print('When predicting the target from the database, we get a ' + str(target_from_database*100)[0:5] + '% accuracy')
    print('When predicting the database from the target, we get a ' + str(database_from_target*100)[0:5] + '% accuracy')
    
    accuracy_roi[0,i]=target_from_database*100
    accuracy_roi[1,i]=database_from_target*100
    # For the figure, we also get self-identifiability and reconstructed self-identifiability
    self_id_roi[i,:]= np.diagonal(sp.stats.zscore(predictions, axis = 1))

print(self_id_roi)

When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When p

When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 79.87% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 84.41% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 85.06% accuracy
When predicting the target from the database, we get a 80.51% accuracy
When predicting the database from the target, we get a 84.41% accuracy
When p

In [13]:
df = pd.DataFrame(self_id_roi)
df.to_csv("AIC_ROI_self_identification_tester.csv")

df = pd.DataFrame(accuracy_roi)
df.to_csv("AIC_ROI_accuracy_tester.csv")



# Remove frequency analysis AIC

In [14]:
# Parameters
n_subs = 154 # Change here to get number of participants! 
n_feats = int(68*439)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,12:451].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat


# Get n subjects: both training and testing datasets
onlyfiles = [f for f in listdir('NEWspectraFUL/') if isfile(join('NEWspectraFUL/', f))]
sub_target = np.zeros([n_subs, (n_feats)+1])
sub_database = np.zeros([n_subs, (n_feats)+1])
iv = 0
it = 0
for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
    sub = 'NEWspectraFUL/' + iFile
    print(sub)
    print(sub[33])
    if sub[33] == 'v':
        sub_target[iv, :] = prune_subject_csv(sub)
        iv += 1
    else:
        sub_database[it, :] = prune_subject_csv(sub)
        it += 1
        

# Correlations can be computed as the dot product between two z-scored vectors
z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
target_from_database = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
database_from_target = accuracy_score(range(n_subs), predictions.argmax(axis = 0))

print('When predicting the target from the database, we get a ' + str(target_from_database*100)[0:5] + '% accuracy')
print('When predicting the database from the target, we get a ' + str(database_from_target*100)[0:5] + '% accuracy')

self_id= np.diagonal(sp.stats.zscore(predictions, axis = 1))
        

NEWspectraFUL/sub_00001_spectrum_training.csv
t
0001
NEWspectraFUL/sub_00001_spectrum_validation.csv
v
0001
NEWspectraFUL/sub_00002_spectrum_training.csv
t
0002
NEWspectraFUL/sub_00002_spectrum_validation.csv
v
0002
NEWspectraFUL/sub_00003_spectrum_training.csv
t
0003
NEWspectraFUL/sub_00003_spectrum_validation.csv
v
0003
NEWspectraFUL/sub_00004_spectrum_training.csv
t
0004
NEWspectraFUL/sub_00004_spectrum_validation.csv
v
0004
NEWspectraFUL/sub_00005_spectrum_training.csv
t
0005
NEWspectraFUL/sub_00005_spectrum_validation.csv
v
0005
NEWspectraFUL/sub_00006_spectrum_training.csv
t
0006
NEWspectraFUL/sub_00006_spectrum_validation.csv
v
0006
NEWspectraFUL/sub_00007_spectrum_training.csv
t
0007
NEWspectraFUL/sub_00007_spectrum_validation.csv
v
0007
NEWspectraFUL/sub_00008_spectrum_training.csv
t
0008
NEWspectraFUL/sub_00008_spectrum_validation.csv
v
0008
NEWspectraFUL/sub_00009_spectrum_training.csv
t
0009
NEWspectraFUL/sub_00009_spectrum_validation.csv
v
0009
NEWspectraFUL/sub_00010_spec

NEWspectraFUL/sub_00077_spectrum_training.csv
t
0077
NEWspectraFUL/sub_00077_spectrum_validation.csv
v
0077
NEWspectraFUL/sub_00078_spectrum_training.csv
t
0078
NEWspectraFUL/sub_00078_spectrum_validation.csv
v
0078
NEWspectraFUL/sub_00079_spectrum_training.csv
t
0079
NEWspectraFUL/sub_00079_spectrum_validation.csv
v
0079
NEWspectraFUL/sub_00080_spectrum_training.csv
t
0080
NEWspectraFUL/sub_00080_spectrum_validation.csv
v
0080
NEWspectraFUL/sub_00081_spectrum_training.csv
t
0081
NEWspectraFUL/sub_00081_spectrum_validation.csv
v
0081
NEWspectraFUL/sub_00082_spectrum_training.csv
t
0082
NEWspectraFUL/sub_00082_spectrum_validation.csv
v
0082
NEWspectraFUL/sub_00083_spectrum_training.csv
t
0083
NEWspectraFUL/sub_00083_spectrum_validation.csv
v
0083
NEWspectraFUL/sub_00084_spectrum_training.csv
t
0084
NEWspectraFUL/sub_00084_spectrum_validation.csv
v
0084
NEWspectraFUL/sub_00085_spectrum_training.csv
t
0085
NEWspectraFUL/sub_00085_spectrum_validation.csv
v
0085
NEWspectraFUL/sub_00086_spec

NEWspectraFUL/sub_00153_spectrum_training.csv
t
0153
NEWspectraFUL/sub_00153_spectrum_validation.csv
v
0153
NEWspectraFUL/sub_00154_spectrum_training.csv
t
0154
NEWspectraFUL/sub_00154_spectrum_validation.csv
v
0154
When predicting the target from the database, we get a 87.01% accuracy
When predicting the database from the target, we get a 88.31% accuracy


In [16]:
print(self_id)

[ 3.47826     2.94226762  2.31849388  2.55495636  0.67555422  4.68379098
  2.24045056  3.44660877  3.49621523  1.78985838  2.41455144  2.93725474
  5.30996298  2.59641128  7.92079852  3.43662163  3.86687643  3.03469871
  4.77819997  2.46750301  2.22199654  2.45019576  5.62440283  2.20334845
  3.19921498  2.2083403   1.27829162  4.5121377   3.04461982  0.42745923
  2.65354051  1.44880251  4.63135712  2.67407341  2.93159912  5.27017065
  2.94292624  2.38494384  2.34037008  3.66989433  3.60465269  3.01735154
  2.88674257  3.54022566  1.854618    6.01206083  2.82368712  6.45852104
  2.26720802  0.32848326  2.94811258  2.97020229  3.02065088  2.73690303
  4.14869472  4.14413804  3.4131515   3.19061106  2.28907377  2.59547945
  4.58739454  2.34543791  4.76077602  4.03312409  4.7971902   4.57679998
  2.68962599  2.65500618  3.47663948  3.55588641  3.02370437  2.82795068
  4.97682223  3.626168    2.90699589  2.47434485  2.50284211  3.57454917
  3.44489287  2.81862673  3.92308086  3.90945678  2

In [15]:
# First we get subject number
def subs_list(sub):
    if sub < 10:
        return 'sub-000' + sub.astype(int).astype(str)
    elif sub >= 10 and sub < 100:
        return 'sub-00' + sub.astype(int).astype(str)
    else: 
        return 'sub-0' + sub.astype(int).astype(str)
    
    
# Get subject data fromCA (Raw)
subs_analyzed = list(map(subs_list, sub_target[:, -1]))
subs_omega = pd.read_csv('dependency/QPN_demo_data.csv', sep=',', header=0)

# Wrangle data to get subjecs' age
sub_moca = list(subs_omega['MoCA (Raw)'].values)
for x in range(len(sub_moca)):
    if isinstance(sub_moca[x], str):
        sub_moca[x] = sub_moca[x][3:5]
    if (sub_moca[x]==""):
        sub_moca[x] = np.nan
        
subs_omega['X.session.moca.'] = sub_moca
subs_omega = subs_omega.rename(columns={'X.session.moca.': 'moca'})

sub_UPDRS = list(subs_omega['UPDRS Score'].values)
for x in range(len(sub_UPDRS)):
    if isinstance(sub_UPDRS[x], str):
        sub_UPDRS[x] = sub_UPDRS[x][3:5]
    if (sub_UPDRS[x]==""):
        sub_UPDRS[x] = np.nan
        
subs_omega['X.session.UPDRS.'] = sub_UPDRS
subs_omega = subs_omega.rename(columns={'X.session.UPDRS.': 'UPDRS'})

subs_omega['self_id_remove_delta'] = self_id

print(self_id)

TypeError: 'list' object is not callable

In [ ]:
# Parameters
n_subs = 154 # Change here to get number of participants! 
n_feats = int(68*439)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,np.append(np.arange(0,12),np.arange(24,451))].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat


# Get n subjects: both training and testing datasets
onlyfiles = [f for f in listdir('NEWspectraFUL/') if isfile(join('NEWspectraFUL/', f))]
sub_target = np.zeros([n_subs, (n_feats)+1])
sub_database = np.zeros([n_subs, (n_feats)+1])
iv = 0
it = 0
for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
    sub = 'NEWspectraFUL/' + iFile
    print(sub)
    print(sub[33])
    if sub[33] == 'v':
        sub_target[iv, :] = prune_subject_csv(sub)
        iv += 1
    else:
        sub_database[it, :] = prune_subject_csv(sub)
        it += 1
        

# Correlations can be computed as the dot product between two z-scored vectors
z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
target_from_database = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
database_from_target = accuracy_score(range(n_subs), predictions.argmax(axis = 0))

print('When predicting the target from the database, we get a ' + str(target_from_database*100)[0:5] + '% accuracy')
print('When predicting the database from the target, we get a ' + str(database_from_target*100)[0:5] + '% accuracy')

self_id= np.diagonal(sp.stats.zscore(predictions, axis = 1))
subs_omega['self_id_remove_theta'] = self_id
        

In [ ]:
# Parameters
n_subs = 154 # Change here to get number of participants! 
n_feats = int(68*436)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,np.append(np.arange(0,24),np.arange(39,451))].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat


# Get n subjects: both training and testing datasets
onlyfiles = [f for f in listdir('NEWspectraFUL/') if isfile(join('NEWspectraFUL/', f))]
sub_target = np.zeros([n_subs, (n_feats)+1])
sub_database = np.zeros([n_subs, (n_feats)+1])
iv = 0
it = 0
for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
    sub = 'NEWspectraFUL/' + iFile
    print(sub)
    print(sub[33])
    if sub[33] == 'v':
        sub_target[iv, :] = prune_subject_csv(sub)
        iv += 1
    else:
        sub_database[it, :] = prune_subject_csv(sub)
        it += 1
        

# Correlations can be computed as the dot product between two z-scored vectors
z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
target_from_database = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
database_from_target = accuracy_score(range(n_subs), predictions.argmax(axis = 0))

print('When predicting the target from the database, we get a ' + str(target_from_database*100)[0:5] + '% accuracy')
print('When predicting the database from the target, we get a ' + str(database_from_target*100)[0:5] + '% accuracy')

self_id= np.diagonal(sp.stats.zscore(predictions, axis = 1))
subs_omega['self_id_remove_alpha'] = self_id
        

In [ ]:
# Parameters
n_subs = 154 # Change here to get number of participants! 
n_feats = int(68*400)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,np.append(np.arange(0,39),np.arange(90,451))].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat


# Get n subjects: both training and testing datasets
onlyfiles = [f for f in listdir('NEWspectraFUL/') if isfile(join('NEWspectraFUL/', f))]
sub_target = np.zeros([n_subs, (n_feats)+1])
sub_database = np.zeros([n_subs, (n_feats)+1])
iv = 0
it = 0
for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
    sub = 'NEWspectraFUL/' + iFile
    print(sub)
    print(sub[33])
    if sub[33] == 'v':
        sub_target[iv, :] = prune_subject_csv(sub)
        iv += 1
    else:
        sub_database[it, :] = prune_subject_csv(sub)
        it += 1
        

# Correlations can be computed as the dot product between two z-scored vectors
z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
target_from_database = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
database_from_target = accuracy_score(range(n_subs), predictions.argmax(axis = 0))

print('When predicting the target from the database, we get a ' + str(target_from_database*100)[0:5] + '% accuracy')
print('When predicting the database from the target, we get a ' + str(database_from_target*100)[0:5] + '% accuracy')

self_id= np.diagonal(sp.stats.zscore(predictions, axis = 1))
subs_omega['self_id_remove_beta'] = self_id
        

In [ ]:
# Parameters
n_subs = 154 # Change here to get number of participants! 
n_feats = int(68*391)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,np.append(np.arange(0,90),np.arange(150,451))].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat


# Get n subjects: both training and testing datasets
onlyfiles = [f for f in listdir('NEWspectraFUL/') if isfile(join('NEWspectraFUL/', f))]
sub_target = np.zeros([n_subs, (n_feats)+1])
sub_database = np.zeros([n_subs, (n_feats)+1])
iv = 0
it = 0
for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
    sub = 'NEWspectraFUL/' + iFile
    print(sub)
    print(sub[33])
    if sub[33] == 'v':
        sub_target[iv, :] = prune_subject_csv(sub)
        iv += 1
    else:
        sub_database[it, :] = prune_subject_csv(sub)
        it += 1
        

# Correlations can be computed as the dot product between two z-scored vectors
z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
target_from_database = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
database_from_target = accuracy_score(range(n_subs), predictions.argmax(axis = 0))

print('When predicting the target from the database, we get a ' + str(target_from_database*100)[0:5] + '% accuracy')
print('When predicting the database from the target, we get a ' + str(database_from_target*100)[0:5] + '% accuracy')

self_id= np.diagonal(sp.stats.zscore(predictions, axis = 1))
subs_omega['self_id_remove_gamma'] = self_id
        

In [ ]:
# Parameters
n_subs = 154 # Change here to get number of participants! 
n_feats = int(68*150)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,0:150].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat


# Get n subjects: both training and testing datasets
onlyfiles = [f for f in listdir('NEWspectraFUL/') if isfile(join('NEWspectraFUL/', f))]
sub_target = np.zeros([n_subs, (n_feats)+1])
sub_database = np.zeros([n_subs, (n_feats)+1])
iv = 0
it = 0
for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
    sub = 'NEWspectraFUL/' + iFile
    print(sub)
    print(sub[33])
    if sub[33] == 'v':
        sub_target[iv, :] = prune_subject_csv(sub)
        iv += 1
    else:
        sub_database[it, :] = prune_subject_csv(sub)
        it += 1
        

# Correlations can be computed as the dot product between two z-scored vectors
z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
target_from_database = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
database_from_target = accuracy_score(range(n_subs), predictions.argmax(axis = 0))

print('When predicting the target from the database, we get a ' + str(target_from_database*100)[0:5] + '% accuracy')
print('When predicting the database from the target, we get a ' + str(database_from_target*100)[0:5] + '% accuracy')

self_id= np.diagonal(sp.stats.zscore(predictions, axis = 1))
subs_omega['self_id_remove_highgamma'] = self_id
        

In [ ]:
subs_omega.to_csv('QPN_demo_with_fingerprinting_score_AIC_analysis_tester.csv')